In [31]:
pip install fake_useragent pandas


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [32]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import pandas as pd


In [33]:
url = 'https://www.imdb.com/chart/boxoffice/?ref_=nv_ch_cht'
ua = UserAgent()
userAgent = ua.random
headers = {'User-Agent': userAgent}
page = requests.get(url, headers = headers)
soup = BeautifulSoup(page.content, 'html.parser')


In [34]:
movie_box = soup.find_all('div', class_ = "ipc-metadata-list-summary-item__c")

titles = []
numbers = []
rate_votes = []
for box in movie_box:
    #titles
    if box.find('h3', class_= "ipc-title__text") is not None:
        title = box.find('h3', class_= "ipc-title__text")
        titles.append(title.text)
    else:
        titles.append('None')

    #numbers
    if box.find('ul', attrs={'data-testid': 'title-metadata-box-office-data-container'}) is not None:
        number = box.find('ul', attrs={'data-testid': 'title-metadata-box-office-data-container'}) 
        numbers.append(number.text)
    else:
        numbers.append('None')


    if box.find('span', attrs={'data-testid': 'ratingGroup--imdb-rating'}) is not None:
        rate_vote = box.find('span', attrs={'data-testid': 'ratingGroup--imdb-rating'}) 
        rate_votes.append(rate_vote.text.replace('\xa0',' '))
    else:
        rate_votes.append('None')


movie_df = pd.DataFrame({
    'title': titles,
    'number': numbers,
    'rate_vote': rate_votes
})

movie_df

,title,number,rate_vote
0,1. Den of Thieves: Pantera,Weekend Gross: $15MTotal Gross: $16MWeeks Rele...,6.6 (2.7K)
1,2. Mufasa: The Lion King,Weekend Gross: $14MTotal Gross: $191MWeeks Rel...,6.7 (29K)
2,3. Sonic the Hedgehog 3,Weekend Gross: $11MTotal Gross: $206MWeeks Rel...,7.2 (26K)
3,4. Nosferatu,Weekend Gross: $6.9MTotal Gross: $83MWeeks Rel...,7.6 (79K)
4,5. Moana 2,Weekend Gross: $6.5MTotal Gross: $435MWeeks Re...,7.0 (63K)
5,6. A Complete Unknown,Weekend Gross: $5.1MTotal Gross: $52MWeeks Rel...,7.8 (12K)
6,7. Wicked: Part I,Weekend Gross: $5.1MTotal Gross: $459MWeeks Re...,7.8 (104K)
7,8. Babygirl,Weekend Gross: $3.1MTotal Gross: $22MWeeks Rel...,6.6 (12K)
8,9. Game Changer,Weekend Gross: $2.1MTotal Gross: $2.1MWeeks Re...,6.8 (24K)
9,10. The Last Showgirl,Weekend Gross: $1.5MTotal Gross: $1.5MWeeks Re...,6.7 (2.4K)


In [35]:
movie_df.number[0]

'Weekend Gross: $15MTotal Gross: $16MWeeks Released: 1'

In [36]:
movie_df['number'] = movie_df['number'].str.replace('Weekend Gross: ','').str.replace('Total Gross: ','|').str.replace('Weeks Released: ','|')
movie_df[['weekend gross', 'total gross','weeks released']] = movie_df['number'].str.split('|', expand = True)
movie_df[['rating','vote_count']] = movie_df['rate_vote'].str.split(' ', expand = True)
movie_df.drop(['number','rate_vote'], axis=1, inplace=True)
movie_df.to_csv('movie_data.csv', index=False)
